In [1]:
import support_functions as sf

sf.generate_very_simple_portfolio_history(no_of_policies=10000, runtime=5)

In [2]:
policy_histories = sf.load_troubled_life_policy_data()
#po licy_histories

In [3]:
policy_histories, history_lengths, max_history_length = sf.prepare_troubled_life_policy_data(policy_histories=policy_histories)
#policy_histories


In [4]:
# Extract features and labels from dataset as numpy.ndarray(s)
policy_histories_labels = policy_histories['troubled'].as_matrix()
policy_histories_features = policy_histories[['premium', 'current_capital']].as_matrix()

# Reshape features from (overall_no_of_histories, 2) to (overall_no_of_policies, maximum_history_length, 2)
policy_histories_features = policy_histories_features.reshape((policy_histories.index.levels[0].shape[0], max_history_length, -1))

# Reshape labels from (overall_no_of_histories) to (overall_no_of_policies, maximum_history_length) and take maximum of each row (0 or 1)
policy_histories_labels = policy_histories_labels.reshape(policy_histories.index.levels[0].shape[0], -1).max(axis=1)

history_lengths = history_lengths[:,1]

#policy_histories_features, policy_histories_labels, history_lengths

In [8]:
import tensorflow as tf
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

tf.reset_default_graph()
tf.set_random_seed(42)

n_steps = max_history_length
n_inputs = 2

n_neurons = 20
n_outputs = 2

learning_rate = 0.001

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# A single RNN layer of n_neurons neurons
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

# We feed the layer's output of the last timestep into a fully connected layer of 2 neurons
# (1 per class, troubled or not). Softmax layer is next
logits = tf.layers.dense(states, n_outputs)
y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#xentropy = y - y_pred
#xentropy = tf.abs(xentropy)

# Loss function and Adam Optimizer
loss = tf.reduce_mean(tf.cast(xentropy, tf.float32))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

loss_summary = tf.summary.scalar('Loss', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

n_epochs = 100
batch_size = 1000

X_batch = policy_histories_features
y_batch = policy_histories_labels
seq_length_batch = history_lengths

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(policy_histories_labels.shape[0] // batch_size):
            X_batch = policy_histories_features[i * batch_size : (i + 1) * batch_size]
            y_batch = policy_histories_labels[i * batch_size : (i + 1) * batch_size]
            seq_length_batch = history_lengths[i * batch_size : (i + 1) * batch_size]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
            #print(outputs.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        #print(logits.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        y_batch_pred = y_pred.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        #acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train)#, "Test accuracy:", acc_test)
        print(loss.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        print(confusion_matrix(y_batch, y_batch_pred))
        print(classification_report(y_batch, y_batch_pred, target_names=['ok', 'troubled']))

        summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        file_writer.add_summary(summary_str, epoch)

file_writer.close()

0 Train accuracy: 0.783
349.036
[[783   0]
 [217   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.88       783
   troubled       0.00      0.00      0.00       217

avg / total       0.61      0.78      0.69      1000

1 Train accuracy: 0.783
48.8602
[[783   0]
 [217   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.88       783
   troubled       0.00      0.00      0.00       217

avg / total       0.61      0.78      0.69      1000



/Users/gopora/MyStuff/Dev/miniconda3/envs/pie35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2 Train accuracy: 0.783
37.3777
[[783   0]
 [217   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.88       783
   troubled       0.00      0.00      0.00       217

avg / total       0.61      0.78      0.69      1000

3 Train accuracy: 0.784
15.3715
[[783   0]
 [216   1]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.88       783
   troubled       1.00      0.00      0.01       217

avg / total       0.83      0.78      0.69      1000



4 Train accuracy: 0.783
20.9285
[[783   0]
 [217   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.88       783
   troubled       0.00      0.00      0.00       217

avg / total       0.61      0.78      0.69      1000

5 Train accuracy: 0.862
5.05843
[[783   0]
 [138  79]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       783
   troubled       1.00      0.36      0.53       217

avg / total       0.88      0.86      0.84      1000

6 Train accuracy: 0.864
3.45561
[[783   0]
 [136  81]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       783
   troubled       1.00      0.37      0.54       217

avg / total       0.88      0.86      0.84      1000



7 Train accuracy: 0.914
2.88703
[[783   0]
 [ 86 131]]
             precision    recall  f1-score   support

         ok       0.90      1.00      0.95       783
   troubled       1.00      0.60      0.75       217

avg / total       0.92      0.91      0.91      1000

8 Train accuracy: 0.853
6.19828
[[783   0]
 [147  70]]
             precision    recall  f1-score   support

         ok       0.84      1.00      0.91       783
   troubled       1.00      0.32      0.49       217

avg / total       0.88      0.85      0.82      1000



...

87 Train accuracy: 1.0
0.00257494
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

88 Train accuracy: 1.0
0.00254463
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

89 Train accuracy: 1.0
0.00251554
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000



90 Train accuracy: 1.0
0.00248696
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

91 Train accuracy: 1.0
0.00245772
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

92 Train accuracy: 1.0
0.00243077
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000



93 Train accuracy: 1.0
0.00240257
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

94 Train accuracy: 1.0
0.00237608
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

95 Train accuracy: 1.0
0.00234905
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000



96 Train accuracy: 1.0
0.00232308
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

97 Train accuracy: 1.0
0.00229802
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

98 Train accuracy: 1.0
0.00227236
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000



99 Train accuracy: 1.0
0.00224822
[[783   0]
 [  0 217]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       783
   troubled       1.00      1.00      1.00       217

avg / total       1.00      1.00      1.00      1000

